<a href="https://colab.research.google.com/github/sojeip/-/blob/main/%ED%8A%B8%EB%A0%88%EC%9D%B8%EC%85%8B_%EC%B5%9C%EC%A2%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import random

np.random.seed(42)
random.seed(42)

# 선수 스탯 부여
pitchers = {
    'P_A': {'ERA': 2.8, 'ERA_RISP': 3.1, 'K_rate': 0.24, 'GB_percent': 0.42, 'IPG': 1.05, 'OPS_against': 0.64, 'OPS_RISP_against':0.67, 'FBP_base':0.3},
    'P_B': {'ERA': 2.9, 'ERA_RISP': 3.0, 'K_rate': 0.30, 'GB_percent': 0.40, 'IPG': 0.95, 'OPS_against': 0.66, 'OPS_RISP_against':0.65, 'FBP_base':0.35},
    'P_C': {'ERA': 3.2, 'ERA_RISP': 3.3, 'K_rate': 0.22, 'GB_percent': 0.38, 'IPG': 0.90, 'OPS_against': 0.68, 'OPS_RISP_against':0.69, 'FBP_base':0.25},
    'P_D': {'ERA': 3.6, 'ERA_RISP': 3.7, 'K_rate': 0.34, 'GB_percent': 0.30, 'IPG': 0.80, 'OPS_against': 0.72, 'OPS_RISP_against':0.70, 'FBP_base':0.4},
    'P_E': {'ERA': 3.1, 'ERA_RISP': 3.3, 'K_rate': 0.20, 'GB_percent': 0.50, 'IPG': 1.20, 'OPS_against': 0.65, 'OPS_RISP_against':0.68, 'FBP_base':0.2}
}

batters = {
    'normal': {'OPS': 0.80, 'OPS_RISP': 0.80},
    'choker': {'OPS': 0.85, 'OPS_RISP': 0.75},
    'clutch': {'OPS': 0.75, 'OPS_RISP': 0.85}
}

runners_codes = ['a','b','c','d','e','f','g','h']  # a: 빈루, b:1루, c:2루, d:3루, e:1,2루, ...

# WPA 계산
def calculate_expected_WPA(p, b, inning, outs, runner_code, score_diff):
    # runner_code -> 간단한 점수화 (none=0, 1루=1, 2루=2, 3루=2, 1,2=3, 1,3=3, 2,3=4, 만루=4)
    runner_map = {'a':0, 'b':1, 'c':2, 'd':2, 'e':3, 'f':3, 'g':4, 'h':4}
    rscore = runner_map.get(runner_code, 0)

    base = 0.0
    # 투수
    base += (- p['ERA']) * 0.03
    base += (- p['ERA_RISP']) * 0.04
    base += p['K_rate'] * 0.5
    base += p['GB_percent'] * 0.1
    base += p['IPG'] * 0.02

    # 타자
    base += b['OPS'] * 0.8
    base += b['OPS_RISP'] * 0.9

    # 경기 상황
    base += (inning - 1) * 0.02
    base -= outs * 0.03
    base += rscore * 0.05
    base -= score_diff * 0.01

    # 노이즈 생성
    noise = round(np.random.normal(0, 0.02), 3)
    return round(base + noise, 3)

# FBP
def compute_fbp(p, inning, score_diff):
    base = p.get('FBP_base', 0.3)
    base += 0.02 * max(0, inning-6)    # 경기 후반일수록 부담이 커지는 경향 고려
    if score_diff <= 0:
        base += 0.05
    # noise
    base += np.random.normal(0, 0.02)
    return round(max(base, 0.0), 3)

# 데이터 생성
rows = []
N_SITUATIONS = 1200  # 상황 수(모든 투수에 대해 생성하므로 행 개수는 X5)
alpha = 0.8          # FBP 계수

for sid in range(1, N_SITUATIONS+1):
    inning_int = random.randint(6, 9)
    outs = random.choice([0,1,2])
    inning = float(f"{inning_int}.{outs}")

    score_diff = random.randint(-3, 4)
    runner = random.choice(runners_codes)
    batter_type = random.choice(list(batters.keys()))
    b = batters[batter_type]

    # 상황마다 모든 투수에 대한 데이터 생성
    for pid, p in pitchers.items():
        # 스탯에 노이즈 첨가
        ERA = round(p['ERA'] + np.random.normal(0, 0.05), 2)
        ERA_RISP = round(p['ERA_RISP'] + np.random.normal(0, 0.05), 2)
        K_rate = round(p['K_rate'] + np.random.normal(0, 0.01), 3)
        GB_percent = round(p['GB_percent'] + np.random.normal(0, 0.02), 3)
        IPG = round(p['IPG'] + np.random.normal(0, 0.05), 2)

        # 상황별 OPS
        OPS = round((p['OPS_against'] + b['OPS']) / 2 + np.random.normal(0, 0.01), 3)
        OPS_RISP = round((p['OPS_RISP_against'] + b['OPS_RISP']) / 2 + np.random.normal(0, 0.01), 3)

        FBP = compute_fbp(p, inning_int, score_diff)

        # WPA, reward
        p_noisy = {'ERA':ERA, 'ERA_RISP':ERA_RISP, 'K_rate':K_rate, 'GB_percent':GB_percent, 'IPG':IPG}
        WPA = calculate_expected_WPA(p_noisy, b, inning_int, outs, runner, score_diff)
        reward = round(WPA - alpha * FBP, 3)

        row = {
            'situation_id': sid,
            'inning': inning,
            'outs': outs,
            'score_diff': score_diff,
            'runners': runner,
            'batter_type': batter_type,
            'pitcher_id': pid,
            'ERA': ERA,
            'ERA_RISP': ERA_RISP,
            'K_rate': K_rate,
            'GB_percent': GB_percent,
            'IPG': IPG,
            'OPS': OPS,
            'OPS_RISP': OPS_RISP,
            'FBP': FBP,
            'WPA': WPA,
            'reward': reward
        }
        rows.append(row)

df_train = pd.DataFrame(rows)

df_train = df_train.round({
    'ERA':2, 'ERA_RISP':2, 'K_rate':3, 'GB_percent':3, 'IPG':2,
    'OPS':3, 'OPS_RISP':3, 'FBP':3, 'WPA':3, 'reward':3
})

# 결과 출력
out_path = "/content/The Trainset.xlsx"
df_train.to_excel(out_path, index=False)
print("trainset_with_IPG.xlsx 생성 완료:", out_path)
print(df_train.head(8))


✅ trainset_with_IPG.xlsx 생성 완료: /content/The Trainset.xlsx
   situation_id  inning  outs  score_diff runners batter_type pitcher_id  \
0             1     6.0     0           1       d      normal        P_A   
1             1     6.0     0           1       d      normal        P_B   
2             1     6.0     0           1       d      normal        P_C   
3             1     6.0     0           1       d      normal        P_D   
4             1     6.0     0           1       d      normal        P_E   
5             2     7.2     2          -2       b      clutch        P_A   
6             2     7.2     2          -2       b      clutch        P_B   
7             2     7.2     2          -2       b      clutch        P_C   

    ERA  ERA_RISP  K_rate  GB_percent   IPG    OPS  OPS_RISP    FBP    WPA  \
0  2.82      3.09   0.246       0.450  1.04  0.718     0.751  0.315  1.522   
1  2.93      2.98   0.295       0.405  0.85  0.713     0.719  0.330  1.554   
2  3.15      3.23   0.